In [1]:
from fileinput import filename
import sys
import os
import six
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from dotenv import load_dotenv
from dotenv import dotenv_values
from pathlib import Path
import logging.config

logging.basicConfig(
    filename="C:\\Users\\Admin\\Documents\\Python\\Selenium\\OkCupid-Automation\\Logs.log")
LOGGER = logging.getLogger()
load_dotenv('.env')


def wait_until_clickable(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.XPATH, xpath)))
    elif class_name:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.CLASS_NAME, class_name)))
    elif el_id:
        WebDriverWait(driver, duration, frequency).until(
            EC.element_to_be_clickable((By.ID, el_id)))


def wait_until_visible(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.XPATH, xpath)))
    elif class_name:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.CLASS_NAME, class_name)))
    elif el_id:
        WebDriverWait(driver, duration, frequency).until(
            EC.visibility_of_element_located((By.ID, el_id)))


def wait_until_present(driver, xpath=None, class_name=None, el_id=None, duration=10000, frequency=0.01):
    if xpath:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elif class_name:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.CLASS_NAME, class_name)))
    elif el_id:
        return WebDriverWait(driver, duration, frequency).until(EC.presence_of_element_located((By.ID, el_id)))



def input_keys(driver, xpath, keys, comment="input_keys"):
    wait_until_clickable(
        driver=driver, xpath=xpath)
    input = driver.find_element_by_xpath(xpath)
    input.clear()
    input.send_keys(keys)
    print(comment)


def press_item(driver, xpath, comment="input_keys"):
    wait_until_clickable(driver=driver, xpath=xpath)
    item = driver.find_element_by_xpath(xpath)
    item.click()
    print(comment)


def initDriver(driver):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.set_page_load_timeout("1")

In [6]:
def login(driver, secret, password):

    print("wait for webpage to open")
    driver.get('chrome-extension://nkbihfbeogaeaoehlefnkodbefgpgknn/home.html#initialize/create-password/import-with-seed-phrase')

    print('Close meta-mask popup')
    chwd = driver.window_handles
    driver.switch_to.window(chwd[0])
    driver.close();
    driver.switch_to.window(chwd[1])

    # Enter form Login for meta mask
    input_keys(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/div[4]/div[1]/div/input', secret, comment="Input secret")
    input_keys(driver, '//*[@id="password"]', password, comment="Input Password")
    input_keys(driver, '//*[@id="confirm-password"]', password, comment="Confirm Password")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/div[7]/div', comment="Check Box")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/form/button', comment="Press Sign in")

    # SHOULD WAIT FOR LOGIN
    time.sleep(1)
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div/button', comment="Select Continue")
    time.sleep(1)

    print("Go to opensea")
    driver.get('https://opensea.io/')
    time.sleep(1)
    press_item(driver, '//*[@id="__next"]/div/div[1]/nav/ul/div[1]/li[4]/a', comment="Select Create")
    press_item(driver, '//*[@id="main"]/div/div/div/div[2]/ul/li[1]', comment="Select Meta Mask Sign in option")
    # Wait for popup sign-in to open
    time.sleep(3)
    chwd = driver.window_handles
    driver.switch_to.window(chwd[1])
    print("Select Meta mask signin buttons")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[2]/div[4]/div[2]/button[2]', comment="Select Meta Mask Sign in option")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[2]/div[2]/div[2]/footer/button[2]', comment="Select Meta Mask Sign in option")
    press_item(driver, '//*[@id="app-content"]/div/div[2]/div/div[3]/button[2]', comment="Select Meta Mask Sign in option")

    print("Success")

# def create(driver):



def run(driver, secret=dotenv_values('.env')['SECRETKEY'], password=dotenv_values('.env')['PASSWORD']):
    maxRetries = 3
    numAttempts = 0
    retryLogin = False

    login(driver, secret, password)
    # create(driver)

    # while numAttempts < maxRetries:
    #     try:
    #         login(driver, secret, password)
    #     except TimeoutException:
    #         LOGGER.info("Failed to login due to timeout. Retrying...")
    #         retryLogin = True
    #     except Exception as e:
    #         LOGGER.exception("Failed to login: " + str(e))
    #         six.reraise(Exception, e, sys.exc_info()[2])
    #     if retryLogin is False and numAttempts < maxRetries:
    #         numAttempts = maxRetries + 1
    #         login(driver, secret, password)
    #     else:
    #         exit()

In [7]:
def chrome():
    executable_path = os.getcwd() + '\\chromedriver.exe'
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe'
    options.add_extension('./10.8.1_0.crx')
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(
        executable_path=executable_path, chrome_options=options)
    run(driver)


def firefox():
    executable_path = os.getcwd() + '\\geckodriver.exe'
    options = webdriver.FirefoxOptions()
    # ADD META EXTENSION
    # options.add_argument("--headless")
    driver = webdriver.Firefox(
        executable_path=executable_path, firefox_options=options)
    run(driver)


chrome()


WebDriverException: Message: unknown error: failed to wait for extension background page to load: chrome-extension://nkbihfbeogaeaoehlefnkodbefgpgknn/background.html
from timeout: Timed out receiving message from renderer: 10.000
